In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

#Plotting Func
def plotx(df, x, y, xlabel=None, ylabel=None, title=None, xticks_range=False, values=None, rotation=None,
         kind=None, decimals=',.0f'):
    """
    Plot a bar chart from a dataframe usind seaborn with bars annotated with values
    -Deps: import seaborn as sns
    """
    
    if (xticks_range and not values) or (values and not xticks_range):
        return f"We will need the values in order to set the xticks_range and vice versa."
    
    try:
        plt.figure(figsize=(20,5))
        #Make the kind of plot specified in 'kind'
        if kind == 'both':
            plots = sns.barplot(x=x,y=y, data=df)
            plots = sns.lineplot(x=x,y=y, data=df)
        elif kind == 'line':
            plots = sns.lineplot(x=x,y=y, data=df)
        else:
            plots = sns.barplot(x=x,y=y, data=df)
            
        if xticks_range and values and rotation:
            plt.xticks(range(xticks_range), values)
            plt.xticks(rotation=rotation)

        elif rotation:
            plt.xticks(rotation=rotation)

        if xlabel:
            plt.xlabel(xlabel)
        if ylabel:
            plt.ylabel(ylabel)
        if title:
            plt.title(title)

        for bar in plots.patches:
            plots.annotate(format(bar.get_height(), decimals),
                       (bar.get_x() + bar.get_width() / 2,
                        bar.get_height()), ha='center', va='center',
                       size=10, xytext=(0, 8),
                       textcoords='offset points')
    except Exception as e:
        print(f"Error while making your plot...:\n{e}")

In [2]:
#Make a chrome browser
#We will keep using this driver to the end
driver = webdriver.Chrome('chromedriver.exe')

In [3]:
#Load the url and wait 10 seconds to finish populating dynamically
url = "https://liveresults.civichive.org/state/ogun?year=2023"
driver.get(url)
sleep(5)

In [4]:
#Grab page contents
res = driver.page_source
info = BeautifulSoup(res,'html.parser') 

In [5]:
#Parse to get all states name for dynamic url building
all_states = [line.text for line in info.find('div', "pageTitle_selectItems__Mc2cy")]
all_states = ['_'.join(line.lower().split()) for line in all_states]
all_states.pop(all_states.index("federal_capital_territory")) #Remove fct full
all_states.append('fct') #replace with fct

In [ ]:
#Get data in a loop
dfs = []
for state in all_states:
    print(f"commencing {state} ....")
    url = f"https://liveresults.civichive.org/state/{state}?year=2023"
    driver.get(url)
    sleep(10)
    
    res = driver.page_source
    info = BeautifulSoup(res,'html.parser')
    try:
        registered_voters = int(info.find('div', "electionCard_footer__b8qQX").h3.text.split(':')[-1].strip().replace(',',''))
        state = info.find('div',"pageTitle_pageTitle__LhfbH").h1.text
    except AttributeError:
        print(f"{state} has no data")
        break
        
    
    #Grab data
    reg = info.find_all('div', {'class':"electionCard_body__NpXzk"})
    rows = []
    for i, line in enumerate(reg[0].find('ul')):
        for item in line:
            if i == 0:
                pass
            else:
                rows.append(item.text)
                
    #arrange
    party = rows[::3]
    totals = rows[2::3]
    percent = rows[1::3]

    df = pd.DataFrame()
    df['Party'] = party
    df['Votes'] = [int(line.strip().replace(',', '')) for line in totals]
    df['Percentage'] = percent
    df["state"] = state
    df["registered voters"] = registered_voters
    dfs.append(df)
    print("done", "\n")
    
    
#Close the browser
driver.quit()

commencing abia ....
done 

commencing adamawa ....
done 

commencing akwa_ibom ....
done 

commencing anambra ....
done 

commencing bauchi ....
done 

commencing bayelsa ....
done 

commencing benue ....
done 

commencing borno ....
done 

commencing cross_river ....
done 

commencing delta ....
done 

commencing ebonyi ....
done 

commencing edo ....
done 

commencing ekiti ....
done 

commencing enugu ....
done 

commencing gombe ....
done 

commencing imo ....
done 

commencing jigawa ....
done 

commencing kaduna ....
done 

commencing kano ....
done 

commencing katsina ....
done 

commencing kebbi ....
done 

commencing kogi ....
done 

commencing kwara ....
done 

commencing lagos ....
done 

commencing nasarawa ....
done 

commencing niger ....
done 

commencing ogun ....
done 

commencing ondo ....
done 

commencing osun ....
done 

commencing oyo ....
done 

commencing plateau ....
done 

commencing rivers ....
done 

commencing sokoto ....
done 

commencing taraba ....
don

In [ ]:
one_table = pd.concat(dfs).reset_index(drop=True)
one_table

In [ ]:
one_table.to_excel("Nigeria Presidential Elections 2023.xlsx", index=False)

In [ ]:
turnout_data = []
state = []
for item in one_table.state.unique():
    state_df = one_table[one_table.state == item].reset_index(drop=True)
    total_voters = state_df.Votes.sum()
    reg_voters = state_df["registered voters"][0]
    turnout = round((total_voters/reg_voters) * 100)
    turnout_data.append(turnout)
    state.append(item.replace('State Results', ''))
    print(f"Percentage turnout {item.replace('State Results', '')} - {turnout}%")

In [ ]:
t_data = pd.DataFrame()
t_data['state'] = state
t_data['turnout'] = turnout_data
t_data = t_data.sort_values('turnout', ascending=False)

In [ ]:
plotx(t_data, 'state', 'turnout', rotation=90, title="Percentage Turnout per State - Nigeria 2023 Presidential Election")

In [ ]:
t_data.plot(figsize=(18, 5), kind='bar', x='state', y="turnout", title="Percentage Turnout per State - Nigeria 2023 Presidential Election");